In [11]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [12]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [13]:
fea_number=7
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_135', 'Input_C_059', 'Input_A5_011', 'Input_C_056',
       'Input_C_053', 'Input_A3_011', 'Input_A2_011'], dtype=object)

## Linear Regression Baseline

In [14]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.0011348670130296502
test_rmse: 0.0011442370831728332
test_r2: 0.762888913734182


## Lasso

In [15]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.0011455083316005306
[0.0009891828415996768, 0.0011267510198912306, 0.0012831399439415872, 0.0010615718659504148, 0.0011672294627778493, 0.001278031888481265, 0.0011002660021856895, 0.0013052211561260792, 0.0010858966368036843, 0.001057792498247831]


test_r2_ave: 0.7650543173640052
[0.8088596236180619, 0.7657447462702572, 0.7517968321708633, 0.8308424013530976, 0.7847115149793393, 0.7247389259500123, 0.7037768863784355, 0.7191328991165982, 0.7739106128296445, 0.7870287309737416]


## ElasticNet

In [16]:
ElasticNet_rmse_ave(skew_data[title],y,[0.008],0.01)

test_rmse_ave: 0.0011619151531082903
[0.0010200766420792506, 0.001127882071265642, 0.0013253486056828926, 0.0011113942618323368, 0.0011871222124048221, 0.0012780009522357948, 0.0010978636817376476, 0.0013228313688907894, 0.0010926846618543723, 0.001055947073099355]


test_r2_ave: 0.758928016399344
[0.7967339282807036, 0.7652742116002362, 0.7351990604150449, 0.8145917686053841, 0.7773107865069462, 0.7247522518163972, 0.7050690211223541, 0.7115027802040559, 0.7710751731791423, 0.7877711822631773]


## SVR

In [17]:
model=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.0011552584765401213
test_rmse: 0.0011443004043795043
test_r2: 0.7636970298066595


## XGB

In [18]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0011255591644585131
[0.0009460494864920952, 0.0010964265120617356, 0.0012739661243948664, 0.0010336909052578648, 0.0011823387412066568, 0.0012736759726907337, 0.0010680446743271046, 0.0013530920585090418, 0.001002408079341646, 0.0010258990903033853]


test_r2_ave: 0.7730050673049105
[0.8251655560767585, 0.7781842004773261, 0.7553332063155976, 0.8396111766541221, 0.7791018089943276, 0.726612072888036, 0.7208726497300073, 0.6981526603370963, 0.8073396622361051, 0.7996776793397282]


## RandomForest

In [19]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 3,
            min_samples_leaf =7,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.001118916309689132
[0.0009605490362739144, 0.0011150928649105627, 0.001338740331599537, 0.0010678791429009514, 0.0011567087653269292, 0.0012317033203886045, 0.0010273334403260274, 0.0012864929526926194, 0.00099859973453773, 0.0010060635079344434]


test_r2_ave: 0.7766919809511834
[0.8197653161248184, 0.7705672060123969, 0.7298207663761055, 0.8288263468683351, 0.7885749839678051, 0.7443336070870843, 0.7417463876005321, 0.7271352183672792, 0.8088007901818656, 0.8073491869256116]


In [20]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-06)
elastic_mod=ElasticNet(alpha=[0.008], l1_ratio=0.01)
svr_mod=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 3,
            min_samples_leaf =7,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0011187082601468103
[0.0009564882786621056, 0.0010869736004899555, 0.00128933704657074, 0.001052385608559309, 0.0011553727740463585, 0.001250893878357293, 0.0010496681189105198, 0.0012874494409576968, 0.001040368587041391, 0.0010181452678727327]


test_r2_ave: 0.776408765970731
[0.8212859929290309, 0.7819925109205348, 0.7493935793646069, 0.8337573275761714, 0.7890630910272137, 0.7363047212454403, 0.7303952317952545, 0.7267293257693482, 0.7924715407410294, 0.8026943383386796]


In [21]:

stack_mod = StackingRegressor(regressors=[svr_mod,random_mod,lasso_mod, vote_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.001175658344069958
[0.0009592560530039205, 0.0011350449088900101, 0.0013112769630442172, 0.0011157734651113128, 0.001193041999027634, 0.0013043397065585292, 0.0011421361969565739, 0.0014003393017683373, 0.0010929400613402661, 0.0011024347849987793]


test_r2_ave: 0.7521976645477744
[0.8202502129404865, 0.7622833999297355, 0.7407921596827689, 0.8131277697375359, 0.775084293718763, 0.7132899936884223, 0.6808025991134938, 0.6767048287615669, 0.7709681448331478, 0.7686732430718228]


In [23]:
ave(skew_data[title],y,0.1,stack_mod,0.6,vote_mod,0.3,random_mod)

test_rmse_ave: 0.0011118938390039244
[0.0009427097172485289, 0.0010805199134094323, 0.0012967972423457343, 0.0010485357729551187, 0.0011484563121941164, 0.0012412352709827467, 0.0010412852894046906, 0.0012871652685947927, 0.001023208048917852, 0.0010090255539862314]


test_r2_ave: 0.7791378714231812
[0.8263977876784194, 0.7845735770430711, 0.7464851362908579, 0.8349714002843927, 0.7915810147057081, 0.7403611745128318, 0.7346842564664174, 0.7268499478309662, 0.7992613051450008, 0.8062131142741461]
